In [4]:
import numpy as np
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer


Assinging data directory and reading files

In [5]:
DATA_DIR="./newsgroup/20_newsgroups/"
folders=sorted(os.listdir(os.path.join(DATA_DIR))) # os.listdir gives a list of all files in this path
folders=folders[1:]

In [6]:
df = pd.DataFrame(columns=['Text','class'])
temp=pd.DataFrame(columns=['Text','class'])

In [7]:
data={}
i=0
#data is a dictionary of the form { folder1 : [doc1,doc2,....,doc1000] , folder2 : [doc1,doc2,doc3,....] }
for folder in folders:
    data[folder]=[]
    for file in os.listdir(os.path.join(DATA_DIR,folder)):
        with open(os.path.join(DATA_DIR,folder,file),encoding='latin-1') as opened_file:
            text=opened_file.read().lower()
            df.loc[i]=[text]+[str(folder)]
        
            i+=1


In [8]:
df

,Text,class
0,newsgroups: alt.atheism\npath: cantaloupe.srv....,alt.atheism
1,newsgroups: alt.atheism\npath: cantaloupe.srv....,alt.atheism
2,path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism
3,path: cantaloupe.srv.cs.cmu.edu!crabapple.srv....,alt.atheism
4,path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism
...,...,...
19989,xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc
19990,xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc
19991,xref: cantaloupe.srv.cs.cmu.edu sci.skeptic:43...,talk.religion.misc
19992,xref: cantaloupe.srv.cs.cmu.edu talk.abortion:...,talk.religion.misc


Intializing stopwords

In [9]:
from nltk.corpus import stopwords # Importing list of stop words from nltk
from string import punctuation # Importing list of punctuations from string
punctuations=list(punctuation)
stopWords=stopwords.words('english')
stopWords+=punctuations # Combined list of stop words

In [10]:
# Common words throughout all docs play no part in classification ,so removing them
stopWords+=['subject:','from:', 'date:', 'newsgroups:', 'message-id:', 'lines:', 'path:', 'organization:', 
            'would', 'writes:', 'references:', 'article', 'sender:', 'nntp-posting-host:', 'people', 
            'university', 'think', 'xref:', 'cantaloupe.srv.cs.cmu.edu', 'could', 'distribution:', 'first', 
            'anyone','world', 'really', 'since', 'right', 'believe', 'still', 
            "max>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'"]

In [36]:
df['cleaned_text']=df['Text'].apply(lambda x: [item for item in x.split() if item not in stopWords])


In [37]:
df

,Text,class,cleaned_text
0,newsgroups: alt.atheism\npath: cantaloupe.srv....,alt.atheism,"[alt.atheism, cantaloupe.srv.cs.cmu.edu!crabap..."
1,newsgroups: alt.atheism\npath: cantaloupe.srv....,alt.atheism,"[alt.atheism, cantaloupe.srv.cs.cmu.edu!das-ne..."
2,path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism,[cantaloupe.srv.cs.cmu.edu!das-news.harvard.ed...
3,path: cantaloupe.srv.cs.cmu.edu!crabapple.srv....,alt.atheism,[cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cm...
4,path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism,[cantaloupe.srv.cs.cmu.edu!das-news.harvard.ed...
...,...,...,...
19989,xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc,"[talk.religion.misc:83766, talk.politics.misc:..."
19990,xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc,"[talk.religion.misc:82815, talk.religion.newag..."
19991,xref: cantaloupe.srv.cs.cmu.edu sci.skeptic:43...,talk.religion.misc,"[sci.skeptic:43397, talk.politics.misc:178808,..."
19992,xref: cantaloupe.srv.cs.cmu.edu talk.abortion:...,talk.religion.misc,"[talk.abortion:120933, talk.religion.misc:8393..."


In [38]:
cleaned_df=pd.DataFrame(df['cleaned_text'])

In [20]:
cleaned_df['text']= cleaned_df['cleaned_text'].apply(lambda x: ' '.join(x))

In [39]:
cleaned_df

,cleaned_text
0,"[alt.atheism, cantaloupe.srv.cs.cmu.edu!crabap..."
1,"[alt.atheism, cantaloupe.srv.cs.cmu.edu!das-ne..."
2,[cantaloupe.srv.cs.cmu.edu!das-news.harvard.ed...
3,[cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cm...
4,[cantaloupe.srv.cs.cmu.edu!das-news.harvard.ed...
...,...
19989,"[talk.religion.misc:83766, talk.politics.misc:..."
19990,"[talk.religion.misc:82815, talk.religion.newag..."
19991,"[sci.skeptic:43397, talk.politics.misc:178808,..."
19992,"[talk.abortion:120933, talk.religion.misc:8393..."


Computing Tf IDF vectors

In [23]:
v = TfidfVectorizer()
x = v.fit_transform(cleaned_df['text'])

Computing Word2Vec Model

In [42]:
cleaned_df1=pd.DataFrame(df['cleaned_text'])

In [43]:
from gensim.models import Word2Vec
from nltk.corpus import gutenberg
from multiprocessing import Pool
from scipy import spatial

In [51]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

model = Word2Vec(sentences=cleaned_df1['cleaned_text'], vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")
model = Word2Vec.load("word2vec.model")
#model.train([["hello", "world"]], total_examples=1, epochs=1)

In [52]:
model.init_sims(replace = True)

/var/folders/fd/_vqm4qqj113dx_9dmjpjrhfw0000gn/T/ipykernel_1166/2675115330.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace = True)


In [53]:
model.wv.most_similar("car")

[('car.', 0.9472602009773254),
 ('head', 0.9437191486358643),
 ('around.', 0.9413886070251465),
 ('bike', 0.9397975206375122),
 ('finish.', 0.9365502595901489),
 ('hot', 0.9364645481109619),
 ('shafts.', 0.9357330799102783),
 ('straight', 0.9356307983398438),
 ('moving', 0.9339444637298584),
 ('tricks', 0.9336975812911987)]

In [54]:
def vectorize(list_of_docs, model):
    
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features
    
vectorized_docs = vectorize(cleaned_df1['cleaned_text'], model=model)
len(vectorized_docs), len(vectorized_docs[0])

(19994, 100)

In [55]:
vectorized_docs

[array([ 0.02637627,  0.0357489 ,  0.01131356,  0.02815813, -0.0465882 ,
        -0.0531438 ,  0.05985713,  0.02847181, -0.06102117, -0.05537418,
         0.06641812, -0.03621873,  0.01892284,  0.04663834,  0.03161744,
        -0.06447771,  0.09037316,  0.00628803, -0.05325022, -0.13087289,
         0.06471816,  0.10147755,  0.17512187,  0.04953997, -0.04484649,
         0.04830106, -0.05345888,  0.0340379 , -0.02898631, -0.01056899,
         0.0926548 ,  0.09900368, -0.05095686, -0.01595202, -0.08344947,
         0.11775509, -0.01122203, -0.1698496 , -0.09906369, -0.06356937,
         0.07956245, -0.12134262,  0.0272016 , -0.06895251,  0.07918493,
         0.08935287, -0.08037941, -0.08982409, -0.02429399, -0.04187413,
         0.06274122, -0.12919262, -0.12732878, -0.01520328, -0.13202764,
        -0.11486034, -0.02435968, -0.0360014 , -0.11234742, -0.05737692,
        -0.02746503, -0.00742334,  0.0654154 , -0.03238222, -0.06445247,
         0.04465283, -0.01459175, -0.07022788, -0.0

Key Phrases Implementation

In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(doc)

print(kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None))